# Generate Functional Connectivities
This script performs further processing on the data by generating the functional connectivities from the preprocessed data. We take in data from the final output of AA in the `aamod_firstlevel_modelestimate_saveresids_00001` folder and generate functional connectivity from the NIfTI files.

## Set up imports

In [21]:
from os import listdir, makedirs
from os.path import isfile, join, exists

from nilearn import plotting, image, datasets
from nilearn.input_data import NiftiSpheresMasker
from nilearn.connectome import ConnectivityMeasure
import numpy as np

## Set up input and output

In [28]:
age_dir = '/imaging3/owenlab/wilson/MovieData/Release8/Age11'
aa_folder = 'BioBank_Analysis_All/'
saveresids_folder = 'aamod_firstlevel_modelestimate_saveresids_00001/'
python_fc_folder = 'python_power_fc'

input_path = join(age_dir, aa_folder, saveresids_folder)
output_path = join(age_dir, aa_folder, python_fc_folder)

# Grab all subjects
subjects = [folder for folder in listdir(input_path) if folder.startswith("sub-")]

## Set up FC functions

In [30]:
power = datasets.fetch_coords_power_2011()
coords = np.vstack((power.rois['x'], power.rois['y'], power.rois['z'])).T

spheres_masker = NiftiSpheresMasker(
    seeds=coords, smoothing_fwhm=6, radius=5.,
    detrend=True, standardize=True, low_pass=0.1, high_pass=0.01, t_r=2)

def generate_power_fc(file):
    time_series = spheres_masker.fit_transform(file)
    
    correlation_measure = ConnectivityMeasure(kind='correlation')
    correlation_matrix = correlation_measure.fit_transform([time_series])[0]
    
    return correlation_matrix

In [5]:
def visualize_corr_matrix(corr_matrix):
    np.fill_diagonal(corr_matrix, 0)
    plotting.plot_matrix(corr_matrix, vmin=-1., vmax=1., colorbar=True, figure=(20, 20),
                     title='Power correlation matrix')
#     plotting.plot_connectome(corr_matrix, coords, title='Power correlation graph',
#                          edge_threshold='98.0%', node_size=20, colorbar=True)

## Generate FCs

In [31]:
for subject in subjects[:1]:
    print("Processing subject:", subject)
    subject_input_path = join(input_path, subject)
    subject_output_path = join(output_path, subject)
    
    # Check if we've already generated the FC for this subject
#     if exists(subject_output_path):
#         print("Skipping subject due to already generated FC.")
#         continue
    
    subject_niftis = [join(subject_input_path, f) for f in listdir(subject_input_path) 
                      if isfile(join(subject_input_path, f)) and f.startswith('ResI')]
    subject_concat_image = image.concat_imgs(subject_niftis)
    
    subject_corr_matrix = generate_power_fc(subject_concat_image)
    
    makedirs(subject_output_path)
    subject_corr_filename = join(subject_output_path, 'power_fc.npy')
    np.save(subject_corr_filename, subject_corr_matrix)
#     visualize_corr_matrix(subject_corr_matrix)

Processing subject: sub-NDAREF074ZAQ
(264, 264)


FileExistsError: [Errno 17] File exists: '/imaging3/owenlab/wilson/MovieData/Release8/Age11/BioBank_Analysis_All/python_power_fc/sub-NDAREF074ZAQ'